## Finetune chinese-llama/alpaca for Tang poem writing

In this notebook, we are going to use Chinese Tang poem data to finetune the Chinese llama/alpaca LLM for poem writing!

### 1. Load Packages

In [ ]:
# Prerequisites
#!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7 transformers[sentencepiece]==4.31.0 scipy tensorboard

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    GenerationConfig,
    HfArgumentParser,
    LlamaTokenizer,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

import warnings
warnings.filterwarnings('ignore')

In [2]:
if torch.cuda.is_available():
    device = torch.device(0)
else:
    device = torch.device('cpu')

print(device)

cuda:0


### 2. Set up model and finetuning parameters

In [24]:
# The model that you want to train from the Hugging Face hub
model_name = "ziqingyang/chinese-alpaca-2-13b"

# The instruction dataset to use
dataset_name = "chenqile09/tang-poems-with-keywords"

# Fine-tuned model name
new_model = "chinese-alpaca-2-13B-tang-poem"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 1000

# Log every X updates steps
logging_steps = 50

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

### 3. Load dataset, tokenizer, and base model from Hugging Face

In [4]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/5274 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36000 [00:00<?, ? examples/s]

In [5]:
# Load tokenizer

tokenizer = LlamaTokenizer.from_pretrained(model_name)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [14]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

print(compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

torch.float16


In [15]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [8]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [16]:
# check GPU memory usage (optional)

!nvidia-smi

Thu Sep 28 02:29:58 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-PCIE-40GB          On  | 00000000:01:00.0 Off |                    0 |
| N/A   31C    P0              36W / 250W |  10761MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

### 4. Load finetuning parameters

In [17]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [25]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [26]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

### 5. Finetune the model

In [27]:
# Train model
trainer.train()

# if the model training stops in the middle, use the following to resume training from the checkpoint
# trainer.train(resume_from_checkpoint="./results/checkpoint-3525")

Step,Training Loss
50,3.005200
100,2.457700
150,2.509000
200,2.520500
250,2.497000
300,2.486800
350,2.477600
400,2.450900
450,2.542400
500,2.465600


TrainOutput(global_step=9000, training_loss=2.3109449827406143, metrics={'train_runtime': 6523.687, 'train_samples_per_second': 5.518, 'train_steps_per_second': 1.38, 'total_flos': 1.2958266472341504e+17, 'train_loss': 2.3109449827406143, 'epoch': 1.0})

In [28]:
# Save trained model
trainer.model.save_pretrained(new_model)

### 6. Merge model in FP16 and merge it with LoRA weights

In [29]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = LlamaTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# need huggingface-cli login
!huggingface-cli login

In [30]:
# push the model and tokenizer to hugging face
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/6.70G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.87G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/844k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/chenqile09/chinese-alpaca-2-13B-tang-poem/commit/d20e8d47b6cdd82b0f8675da1ca115cf7772819a', commit_message='Upload tokenizer', commit_description='', oid='d20e8d47b6cdd82b0f8675da1ca115cf7772819a', pr_url=None, pr_revision=None, pr_num=None)

### 7. Set up eval configs

In [31]:
from tqdm import tqdm
import evaluate
import numpy as np

# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

In [ ]:
# # Load tokenizer and model with QLoRA configuration
# compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# print(compute_dtype)

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=use_4bit,
#     bnb_4bit_quant_type=bnb_4bit_quant_type,
#     bnb_4bit_compute_dtype=compute_dtype,
#     bnb_4bit_use_double_quant=use_nested_quant,
# )

In [ ]:
# # Load base model
# model = AutoModelForCausalLM.from_pretrained(
#     'chenqile09/chinese-alpaca-2-LoRA-13B-couplet-100k',
#     quantization_config=bnb_config,
#     device_map=device_map
# )
# model.config.use_cache = False
# model.config.pretraining_tp = 1

In [ ]:
# # Load base model
# model = AutoModelForCausalLM.from_pretrained(
#     new_model,
#     quantization_config=bnb_config,
#     device_map=device_map
# )
# model.config.use_cache = False
# model.config.pretraining_tp = 1

In [ ]:
# # Load tokenizer
# tokenizer = LlamaTokenizer.from_pretrained('chenqile09/chinese-alpaca-2-LoRA-13B-couplet-100k')

In [50]:
# Set up the text generation config

generation_config = GenerationConfig(
    temperature=0.2,
    top_k=40,
    top_p=0.9,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.1,
    max_new_tokens=1000
)

In [51]:
SYSTEM_PROMPT = """你是一个写唐诗的AI助手"""

TEMPLATE = (
    "<s>[INST] <<SYS>> {system_prompt} <</SYS>> 请以{keywords}为主题，作一首唐诗。[/INST]"
)

def generate_prompt(keywords_list, system_prompt=SYSTEM_PROMPT):
    return [TEMPLATE.format_map({'keywords': keywords,'system_prompt': system_prompt}) for keywords in keywords_list]

def generate_output_from_llm(model, tokenizer, generation_config, keywords, system_prompt=SYSTEM_PROMPT):

    input_texts = generate_prompt(keywords, system_prompt)
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True)
    generation_outputs = model.generate(
        input_ids = inputs["input_ids"].to(device),
        attention_mask = inputs['attention_mask'].to(device),
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        generation_config = generation_config
    )

    outputs = [tokenizer.decode(s, skip_special_tokens=True) for s in generation_outputs]
    responses = [output.split("[/INST]")[-1].strip() for output in outputs]
    return responses

In [56]:
# some example inputs
keywords_list = [
    "天涯，芳草，思念",
    "闲日，无年，高树",
    "上海，武汉，思念",
    "春日，悠闲，快乐",
    "帝王将相，围猎，壮志未酬",
    "佛，心，亲",
    "猫咪，亲昵，思念",
    "粪坑，眼镜，裤子",
    "灯光，幻象，女人"
]

In [57]:
# Generate responses in batch, and then print out

responses = generate_output_from_llm(
    model,
    tokenizer,
    generation_config,
    keywords_list,
    SYSTEM_PROMPT
)

for keywords, response in zip(keywords_list, responses):
    print(keywords+'\n'+response+'\n')

天涯，芳草，思念
南国天涯去不还，芳草萋萋绿满山。回首望乡心更苦，不知何处是家关。

闲日，无年，高树
闲日何曾有，无年亦不长。高树常相望，流水自潺湲。

上海，武汉，思念
汉水东流入大海，楚人南望见吴台。君王欲问何处去？妾家江上一帆开。

春日，悠闲，快乐
柳絮飘飞满院香，莺啼燕语入帘长。何当踏破青苔去，笑向东园看牡丹。

帝王将相，围猎，壮志未酬
汉家天子出龙门，百辟随从入苑前。将军若有雄心在，何须陪君到此边。

佛，心，亲
无相不离佛，有相是心身。若能见此理，即为佛与亲。

猫咪，亲昵，思念
猫儿爱睡时，爪尖枕上起。若见主人归，便来卧膝侧。

粪坑，眼镜，裤子
粪坑是何物？玻璃似眼镜。裤裆如铜钱，不须穿裤子。

灯光，幻象，女人
灯下人影似仙郎，画屏红烛照双光。不知是梦还是真，一曲笙歌欲晓长。



In [ ]:
# 天涯，芳草，思念
# 南国天涯去不还，芳草萋萋绿满山。回首望乡心更苦，不知何处是家关。

# 闲日，无年，高树
# 闲日何曾有，无年亦不长。高树常相望，流水自潺湲。

# 上海，武汉，思念
# 汉水东流入大海，楚人南望见吴台。君王欲问何处去？妾家江上一帆开。

# 春日，悠闲，快乐
# 柳絮飘飞满院香，莺啼燕语入帘长。何当踏破青苔去，笑向东园看牡丹。

# 帝王将相，围猎，壮志未酬
# 汉家天子出龙门，百辟随从入苑前。将军若有雄心在，何须陪君到此边。

# 佛，心，亲
# 无相不离佛，有相是心身。若能见此理，即为佛与亲。

# 猫咪，亲昵，思念
# 猫儿爱睡时，爪尖枕上起。若见主人归，便来卧膝侧。

# 粪坑，眼镜，裤子
# 粪坑是何物？玻璃似眼镜。裤裆如铜钱，不须穿裤子。

# 灯光，幻象，女人
# 灯下人影似仙郎，画屏红烛照双光。不知是梦还是真，一曲笙歌欲晓长。